This notebook demonstrates how to link files into a Galaxy data library. The library must exist for this to work. This notebook uses [bioblend](https://github.com/galaxyproject/bioblend). It's necessary to have a Galaxy API key, which is read in from a file in the home directory.

In [1]:
from bioblend.galaxy import GalaxyInstance
import sys
import os

f = open("{}/.galaxy_key".format(os.path.expanduser("~")))
userKey = f.readline().strip()
f.close()
url = "http://pc331.immunbio.mpg.de:8888"

gi = GalaxyInstance(url=url, key=userKey)

These are the folders the currently exist in the `foo bar` library.

In [2]:
folders = gi.libraries.get_folders(gi.libraries.get_libraries(name="foo bar")[0]["id"])
print("{} folders".format(len(folders)))
for k, v in folders[0].items():
    print("{}: {}".format(k, v))

1 folders
name: /
url: /api/libraries/f2db41e1fa331b3e/contents/Ff2db41e1fa331b3e
id: Ff2db41e1fa331b3e
type: folder


`addFileToLibrary()` is a wrapper around all of the other functions is is probably the one to directly use.

In [3]:
def getLibID(gi, libName):
    """
    Given a library name, like "foo bar", return the ID for the first library matching that
    """
    lib = gi.libraries.get_libraries(name=libName)
    if not lib or len(lib) == 0:
        raise RuntimeError("No library named {}".format(libName))
    return lib[0]["id"]


def getFolderID(gi, libID, path):
    """
    Given a library ID (lib) and a path (e.g., "/foo/bar/sniggly3"), return the folder ID for sniggly3.
    
    If the path doesn't exist (in part or in total), then create it.
    """
    # Does the path already exist?
    folders = gi.libraries.get_folders(libID, name=path)
    if folders is not None and len(folders) > 0:
        return folders[0]["id"]
    
    # Get the closest base folder
    longest = gi.libraries.get_folders(libID, name="/")[0]
    folders = gi.libraries.get_folders(libID)
    for folder in folders:
        if path.startswith(folder["name"]):
            # Look for the longest pathname overlap. The next character in path MUST be "/",
            # since otherwise adding "/a/b/c2" when "/a/b/c" exists would result in "/a/b/c/2"!
            if len(folder["name"]) > len(longest["name"]) and path[len(folder["name"])] == "/":
                longest = folder
    
    # shorten the path name if relevant
    idx = len(longest["name"])
    pathLeft = path[idx:]
    if pathLeft.startswith("/"):
        pathLeft = pathLeft[1:]

    for fName in pathLeft.split("/"):
        gi.libraries.create_folder(libID, fName, base_folder_id=longest["id"]) # returns None
        if longest["name"] != "/":
            newFName = "{}/{}".format(longest["name"], fName)
        else:
            newFName = "/{}".format(fName)
        longest = gi.libraries.get_folders(libID, name=newFName)[0]
    
    return longest["id"]


def addFileToLibraryFolder(gi, libID, folderID, fileName, file_type='auto', dbkey='?', link=True, roles=''):
    """
    Link/copy "fname" into the library and folder specified by libID and folderID. These MUST exist.
    
    file_type, dbkey, and roles are pass through to upload_from_galaxy_filesystem().
    
    link must be True (default) or False. If it's False then files are copied in.
    
    This returns a dictionary with keys: name, url, and id (or presumably None on error).
    """
    if link == True:
        link_data_only = 'link_to_files'
    else:
        link_data_only = 'copy_files'
    
    try:
        rv = gi.libraries.upload_from_galaxy_filesystem(libID,
                                                        fileName,
                                                        folder_id=folderID,
                                                        file_type=file_type,
                                                        dbkey=dbkey,
                                                        link_data_only=link_data_only,
                                                        roles=roles)
        return rv
    except:
        sys.stderr.write("Received the following error while adding '{}': '{}'".format(fileName, sys.exc_info()[0]))
        return None


def addFileToLibrary(gi, libraryName, path, fileName, file_type='auto', dbkey='?', link=True, roles=''):
    """
    Add fileName to path in libraryName. gi is a GalaxyInstance
    
    The other parameters are passed to upload_from_galaxy_filesystem()
    """
    libID = getLibID(gi, libraryName)
    folderID = getFolderID(gi, libID, path)
    dataset = addFileToLibraryFolder(gi, libID, folderID, fileName, file_type=file_type, dbkey=dbkey, link=link, roles='')
    if not dataset:
        raise RuntimeError("Error adding '{}' to '{}'".format(fileName, path))
    return dataset

The most common "file_type"s will be:
    * bam
    * bigwig
    * fastqsanger
    * tabular

Below I'll add a bigWig file ("/data/python2.bw" on the server) to "foo bar":"/foo/bar/sniggly" and "foo bar":"/foo/bar/sniggly2". We'll add it with different file types to demonstrate how to do that.

In [4]:
dataset = addFileToLibrary(gi, "foo bar", "/foo/bar/sniggly", "/data/python2.bw")
print(dataset)
dataset = addFileToLibrary(gi, "foo bar", "/foo/bar/sniggly2", "/data/python2.bw", file_type="fastqsanger")
print(dataset)
dataset = addFileToLibrary(gi, "foo bar", "/foo/bar/sniggly", "/data/python2.bw", file_type="fastqsanger")
print(dataset)

[{'name': 'python2.bw', 'url': '/api/libraries/f2db41e1fa331b3e/contents/417e33144b294c21', 'id': '417e33144b294c21'}]
[{'name': 'python2.bw', 'url': '/api/libraries/f2db41e1fa331b3e/contents/2d9035b3fc152403', 'id': '2d9035b3fc152403'}]
[{'name': 'python2.bw', 'url': '/api/libraries/f2db41e1fa331b3e/contents/5a1cff6882ddb5b2', 'id': '5a1cff6882ddb5b2'}]


These are now the folders that exist in "foo bar":

In [5]:
libID = getLibID(gi, "foo bar")
for l in gi.libraries.get_folders(libID):
    print(l)

{'name': '/', 'url': '/api/libraries/f2db41e1fa331b3e/contents/Ff2db41e1fa331b3e', 'id': 'Ff2db41e1fa331b3e', 'type': 'folder'}
{'name': '/foo', 'url': '/api/libraries/f2db41e1fa331b3e/contents/Fba03619785539f8c', 'id': 'Fba03619785539f8c', 'type': 'folder'}
{'name': '/foo/bar', 'url': '/api/libraries/f2db41e1fa331b3e/contents/Fcbbbf59e8f08c98c', 'id': 'Fcbbbf59e8f08c98c', 'type': 'folder'}
{'name': '/foo/bar/sniggly', 'url': '/api/libraries/f2db41e1fa331b3e/contents/F964b37715ec9bd22', 'id': 'F964b37715ec9bd22', 'type': 'folder'}
{'name': '/foo/bar/sniggly2', 'url': '/api/libraries/f2db41e1fa331b3e/contents/F1fad1eaf5f4f1766', 'id': 'F1fad1eaf5f4f1766', 'type': 'folder'}
